In [2]:
import sys
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
import numpy as np
import random

In [7]:
path = "data/raw/"
essay_file = path+ "asap-aes/train.tsv"

i=0

essayList = []
with open(essay_file) as f:
    for line in f:
        if line.startswith("Id"):
            continue
        splitLine = line.strip().split("\t")
        scores = map(int, splitLine[2:4])
        avgscore = sum(scores)/2
        essay = sent_tokenize(splitLine[4])
        for sent in essay:
            essayList.append([sent, avgscore])
    

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/asap-aes/train.tsv'

In [10]:
path = "data/raw/"
short_answer_file = path+ "asap-sas/train.tsv"

i=0

dataList = []
with open(short_answer_file) as f:
    for line in f:
        if line.startswith("Id"):
            continue
        splitLine = line.strip().split("\t")
        scores = map(int, splitLine[2:4])
        avgscore = sum(scores)/2
        essay = sent_tokenize(splitLine[4])
        for sent in essay:
            dataList.append([sent, avgscore])
    

In [11]:
highScores = []
lowScores = []
for data in dataList:
    if len(word_tokenize(data[0])) > 20:
        continue
    if data[1] >= 2:
        highScores.append(data[0])
    if data[1] ==0:
        lowScores.append(data[0])
    

In [40]:
import csv

with open('sas_high.csv', 'w', newline='\n') as f:
    writer = csv.writer(f)
    writer.writerows(highScores)
    

with open('sas_low.csv', 'w', newline='\n') as f:
    writer = csv.writer(f)
    writer.writerows(lowScores)

In [42]:
Counter([x[1] for x in dataList])

Counter({1.0: 11279,
         0.0: 11879,
         2.0: 11669,
         0.5: 2255,
         3.0: 2247,
         2.5: 510,
         1.5: 3216})

In [131]:

random.shuffle(cleanLow)
train_low = cleanLow[:5000]
test_low = cleanLow[5500:]
dev_low = cleanLow[5000:5500]

random.shuffle(cleanHigh)
train_high = cleanHigh[:5000]
test_high = cleanHigh[5500:]
dev_high = cleanHigh[5000:5500]

In [23]:
import string

def clean(data_list):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    newList = []
    for sent in data_list:
        if len(sent) < 40:
            continue
        if '^' in sent:
            continue
        #sent = re.sub(r'\d+', '', sent)
        

        #sent = re.sub('['+string.punctuation+']', '', sent)
        sent = sent.lower()
        
        newList.append(sent + '\n')
    
    return newList
        

In [24]:
cleanLow = clean(lowScores)

In [25]:
cleanHigh = clean(highScores)

In [26]:
len(cleanLow)

6057

In [27]:
len(cleanHigh)

7221

In [28]:
with open('sas_data.1', 'w') as f:
    f.writelines(cleanHigh)

In [29]:
with open('sas_data.0', 'w') as f:
    f.writelines(cleanLow)

In [106]:
high_label = [1 for x in cleanHigh]
low_label = [0 for x in cleanLow]

In [111]:
labels = high_label + low_label

In [113]:
all_data = cleanHigh + cleanLow

In [114]:
import random

c = list(zip(all_data, labels))

random.shuffle(c)

shuff_data, shuff_labels = zip(*c)


In [119]:
with open('../text_style_transfer/model/data/sas_data', 'w') as f:
    retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', shuff_data)
    f.writelines(retokenized)
    
# with open('../text_style_transfer/model/data/sas.test.1', 'w') as f:
#     retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', test_high)
#     f.writelines(retokenized)
    
# with open('../text_style_transfer/model/data/sas.dev.1', 'w') as f:
#     retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', dev_high)
#     f.writelines(retokenized)

with open('../text_style_transfer/model/data/sas_labels', 'w') as f:
    f.writelines(map(lambda x: str(x) + '\n', shuff_labels))

In [132]:


with open('../language-style-transfer/data/sas.train.1', 'w') as f:
    retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', train_high)
    f.writelines(retokenized)
    
with open('../language-style-transfer/data/sas.test.1', 'w') as f:
    retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', test_high)
    f.writelines(retokenized)
    
with open('../language-style-transfer/data/sas.dev.1', 'w') as f:
    retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', dev_high)
    f.writelines(retokenized)

with open('../language-style-transfer/data/sas.train.0', 'w') as f:
    retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', train_low)
    f.writelines(retokenized)
    
with open('../language-style-transfer/data/sas.test.0', 'w') as f:
    retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', test_low)
    f.writelines(retokenized)
    
with open('../language-style-transfer/data/sas.dev.0', 'w') as f:
    retokenized = map(lambda x: ' '.join(word_tokenize(x)) + '\n', dev_low)
    f.writelines(retokenized)
    

In [53]:
' '.join(word_tokenize(highScores[0][0])) 

'You would need many more pieces of information to replicate the experiment .'

In [ ]:
def sentence_to_wordlist(raw_sentence):
    
    clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
    tokens = nltk.word_tokenize(clean_sentence)
    
    return tokens

In [ ]:

def tokenize(essay):
    stripped_essay = essay.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokenized_sentences.append(sentence_to_wordlist(raw_sentence))
    
    return tokenized_sentences

In [47]:
clean_text(dataList[0][0])

'some additional information that we would need to replicate the experiment is how much vinegar should be placed in each identical container how or what tool to use to measure the mass of the four different samples and how much distilled water to use to rinse the four samples after taking them out of the vinegar'

In [ ]:
class Vocabulary(object):
    def __init__(self, vocab_file, emb_file='', dim_emb=0):
        with open(vocab_file, 'rb') as f:
            self.size, self.word2id, self.id2word = pickle.load(f)
        self.dim_emb = dim_emb
        self.embedding = np.random.random_sample(
            (self.size, self.dim_emb)) - 0.5

        if emb_file:
            print( 'Loading word vectors from', emb_file)
            with open(emb_file) as f:
                for line in f:
                    parts = line.split()
                    word = parts[0]
                    vec = np.array([float(x) for x in parts[1:]])
                    if word in self.word2id:
                        self.embedding[self.word2id[word]] = vec

        for i in range(self.size):
            self.embedding[i] /= LA.norm(self.embedding[i])

def build_vocab(data, path, min_occur=5):
    word2id = {'<pad>':0, '<go>':1, '<eos>':2, '<unk>':3}
    id2word = ['<pad>', '<go>', '<eos>', '<unk>']

    words = [word for sent in data for word in sent]
    cnt = Counter(words)
    for word in cnt:
        if cnt[word] >= min_occur:
            word2id[word] = len(word2id)
            id2word.append(word)
    vocab_size = len(word2id)
    with open(path, 'wb') as f:
        pickle.dump((vocab_size, word2id, id2word), f, pickle.HIGHEST_PROTOCOL)


In [152]:
with open('../language-style-transfer/data/sas.test.0') as f:
    data_0 = f.readlines()

In [151]:
with open('../language-style-transfer/data/sas.test.1') as f:
    data_1 = f.readlines()

In [133]:
with open('../language-style-transfer/tmp/sas.test.0.tsf') as f:
    test_0 = f.readlines()

In [134]:
with open('../language-style-transfer/tmp/sas.test.1.tsf') as f:
    test_1 = f.readlines()

In [145]:
def strip_unk(sent):
    sent = re.sub(r'<unk>', '', sent)
    
    return word_tokenize(sent)

In [155]:
test_1_strip = list(map(strip_unk, test_1))
test_0_strip = list(map(strip_unk, test_0))
data_1_strip = list(map(strip_unk, data_1))
data_0_strip = list(map(strip_unk, data_0))

In [157]:
def bleu(sent1, sent2):
    
    score = nltk.translate.bleu_score.sentence_bleu([sent1], sent2)
    return score

In [158]:

bleus_1 = []
for i in range(len(test_1_strip)):
    bleus_1.append(bleu(data_1_strip[i],test_1_strip[i]))

/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
The

In [160]:
data_1_strip[0],test_1_strip[0]

(['the',
  'author',
  'first',
  'starts',
  'out',
  'with',
  'the',
  'introduction',
  'which',
  'explains',
  'what',
  'it',
  'is',
  'about'],
 ['the',
  'author',
  'organizes',
  'the',
  'article',
  'in',
  'what',
  'it',
  'is',
  'going',
  'about',
  'space',
  'junk'])

In [162]:
data_1_strip[1],test_1_strip[1]

(['as',
  'the',
  'article',
  'states',
  'in',
  'paragraph',
  'the',
  'snakes',
  'have',
  'imperiled',
  'endangered',
  'species',
  'in',
  'the',
  'florida',
  'keys'],
 ['in', 'the', 'and', 'the', 'koala', 'are', 'different', 'of', 'the'])